In [1]:
import numpy as np
import pandas as pd

from datasets import load_from_disk

In [2]:
qa_dataset = load_from_disk('/home/ddz5/Desktop/cell2gsea/gene_program_finetuning_nbs/hf_dataset_2024-12-06')

In [74]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 566
    })
    validation: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 105
    })
    test: Dataset({
        features: ['Context', 'cell_sentences_data', 'Question', 'Answer', 'Keyword', 'full_QA_pair', 'Publication_Title', 'Dataset_URL', 'Dataset_Index', 'Used_Rows', 'openai_batch_response_id'],
        num_rows: 99
    })
})

In [77]:
len(set(qa_dataset['train']['Publication_Title'])) + len(set(qa_dataset['validation']['Publication_Title'])) + len(set(qa_dataset['test']['Publication_Title']))

26

In [54]:
dataset_local_indices = set(qa_dataset['train']['Dataset_Index']) | set(qa_dataset['test']['Dataset_Index']) | set(qa_dataset['validation']['Dataset_Index'])

In [56]:
len(dataset_local_indices)

26

In [78]:
import os

In [42]:
match_datasets = pd.read_csv('/gpfs/radev/home/sz568/scratch/C2S_RL/depose/matching_datasets.csv')

In [43]:
match_datasets

,dataset,filename
0,Activated PBMCs - Expanded cohort CITE-seq,local(304).h5ad
1,blood and bone marrow from a healthy young donor,local(477).h5ad
2,scRNA-seq analysis of healthy human fallopian ...,local(770).h5ad
3,HypoMap – a unified single cell gene expressio...,local(52).h5ad
4,L4 IT - MTG: Seattle Alzheimer's Disease Atlas...,local(274).h5ad
...,...,...
86,Molecular characterization of selectively vuln...,local(604).h5ad
87,Single-cell longitudinal analysis of SARS-CoV-...,local(778).h5ad
88,Atlas of the developing mouse thalamus,local(466).h5ad
89,Colon,local(777).h5ad


In [64]:
match_datasets['filename'][0].split('local(')[1].split(').h5ad')[0]

'304'

In [71]:
all_local_indices = []
for filename in match_datasets['filename']:
    all_local_indices.append(filename.split('local(')[1].split(').h5ad')[0])

In [72]:
len(all_local_indices)

91

In [73]:
num_datasets = 0
for local_indices in list(dataset_local_indices):
    if local_indices not in all_local_indices:
        num_datasets += 1
        print(local_indices)

678
503
593


In [68]:
num_datasets

26

In [46]:
match_datasets['dataset'].nunique()

91

In [7]:
used_datasets = np.load('/home/sz568/scratch/C2S_RL/depose/overlapping_datasets.npy')

In [41]:
used_datasets

array(['Activated PBMCs - Expanded cohort CITE-seq',
       'blood and bone marrow from a healthy young donor',
       'scRNA-seq analysis of healthy human fallopian tubes',
       'HypoMap – a unified single cell gene expression atlas of the murine hypothalamus',
       "L4 IT - MTG: Seattle Alzheimer's Disease Atlas (SEA-AD)",
       'Lung Parenchyma',
       'Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease: SFG astrocytes',
       'Human kidney cortex snATAC-seq data from donors with and without diabetic kidney disease',
       'fetal', 'healthy young bone marrow donor',
       'A Single-Cell Transcriptional Roadmap of the Mouse and Human Lymph Node Lymphatic Vasculature - mouse',
       'e14.5 glutamatergic lineage',
       "VLMC - MTG: Seattle Alzheimer's Disease Atlas (SEA-AD)",
       'e16.5 glutamatergic lineage',
       "Sst Chodl - MTG: Seattle Alzheimer's Disease Atlas (SEA-AD)",
       'Bipolar cells of the human fovea and peripheral reti

In [26]:
for dataset_name in used_datasets:
    if dataset_name not in match_datasets['dataset'].values:
        print(dataset_name)